In [1]:
import pandas as pd

df = pd.read_csv('data/bank-full.csv', sep=';')

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [2]:
df.shape

(45211, 17)

### Check data types

In [3]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

### Check duplicated rows

In [4]:
df[df.duplicated()]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y


### Check missing values

In [5]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [6]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df,title="Bank Marketing")

profile.to_file("banking_report.html")

c:\Users\macie\anaconda3\envs\solvroml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 110.66it/s]


### Drop 'Duration' Col

After initial analysis there is no point in training model on this variable, since  we want to predict whether the customer will open lokata before we call them.

In [7]:
df = df.drop(columns='duration')

df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

### Datatype Conversion

Now we convert the data types for model to handle.

Rule of conversion:
1. Numeric - no conversion
2. Categorical - preserve relationships where necessary (e.g month, but not job)
3. Boolean - convert to binary

In [8]:
# categorical
    # Education mapping
edu_map = {'unknown': 0, 'primary': 1, 'secondary': 2, 'tertiary': 3}
df['education'] = df['education'].map(edu_map)

    # Month mapping
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_map = {m: i+1 for i, m in enumerate(months)}
df['month'] = df['month'].map(month_map)

# One Hot encoding
df = pd.get_dummies(df, columns=['job', 'marital', 'contact', 'poutcome'], drop_first=True)

# binary
binary_cols = ['default', 'housing', 'loan', 'y']
for col in binary_cols:
    df[col] = df[col].map({'yes': 1, 'no': 0})

df.dtypes # print result

age                  int64
education            int64
default              int64
balance              int64
housing              int64
loan                 int64
day                  int64
month                int64
campaign             int64
pdays                int64
previous             int64
y                    int64
job_blue-collar       bool
job_entrepreneur      bool
job_housemaid         bool
job_management        bool
job_retired           bool
job_self-employed     bool
job_services          bool
job_student           bool
job_technician        bool
job_unemployed        bool
job_unknown           bool
marital_married       bool
marital_single        bool
contact_telephone     bool
contact_unknown       bool
poutcome_other        bool
poutcome_success      bool
poutcome_unknown      bool
dtype: object

### One More Profiling

Let's create one more report to see how the data changed, mostly visually, since we have more columns now, and how the alerts look like

In [9]:
profile = ProfileReport(df,title="Bank Marketing 2")

profile.to_file("banking_report_v2.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 99.99it/s]


In [10]:
# Check unique values to spot differences
print("Education unique values:", df['education'].unique())
print("Month unique values:", df['month'].unique())

df.head()

Education unique values: [3 2 0 1]
Month unique values: [ 5  6  7  8 10 11 12  1  2  3  4  9]


,age,education,default,balance,housing,loan,day,month,campaign,pdays,...,job_technician,job_unemployed,job_unknown,marital_married,marital_single,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,58,3,0,2143,1,0,5,5,1,-1,...,False,False,False,True,False,False,True,False,False,True
1,44,2,0,29,1,0,5,5,1,-1,...,True,False,False,False,True,False,True,False,False,True
2,33,2,0,2,1,1,5,5,1,-1,...,False,False,False,True,False,False,True,False,False,True
3,47,0,0,1506,1,0,5,5,1,-1,...,False,False,False,True,False,False,True,False,False,True
4,33,0,0,1,0,0,5,5,1,-1,...,False,False,True,False,True,False,True,False,False,True


In [11]:
df = df.drop(columns='marital_single')
df.dtypes

age                  int64
education            int64
default              int64
balance              int64
housing              int64
loan                 int64
day                  int64
month                int64
campaign             int64
pdays                int64
previous             int64
y                    int64
job_blue-collar       bool
job_entrepreneur      bool
job_housemaid         bool
job_management        bool
job_retired           bool
job_self-employed     bool
job_services          bool
job_student           bool
job_technician        bool
job_unemployed        bool
job_unknown           bool
marital_married       bool
contact_telephone     bool
contact_unknown       bool
poutcome_other        bool
poutcome_success      bool
poutcome_unknown      bool
dtype: object

### EDA Summary

The dataset is pretty clean for modelling. 'Duration' column was dropped to prevent data leak'age. There were no null values. Outliers were not analyzed nor handled. The variable 'y' will be hanlded in thr iraining loop and in the model testing - as we will use precision instead of accuracy. 'marital_single' was dropped due to presence of 'marital_married', which couples singles with divorced people but it should be fine.

# Preprocessing

In [12]:
features = df.drop(columns='y')
target = df['y']

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)



In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(class_weight='balanced', random_state=42, )

In [15]:
classifier.fit(x_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,'balanced'


In [16]:
y_pred = classifier.predict(x_test)

In [17]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_pred=y_pred, y_true=y_test)
recall = recall_score(y_pred=y_pred, y_true=y_test)

In [18]:
print("Precision: ", precision)
print("Recall: ", recall)

Precision:  0.2849557522123894
Recall:  0.29514207149404215


## Decision Tree Model Summary

Out of those customers who said yes, model predicted ~28% correctly, the rest are false positives.
The model missed out on ~70% customers who would've said yes.

In [29]:
from model_utils import ModelTester
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

tester = ModelTester(x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

# Decision Tree models with different hyperparameters
dt_depth_5 = DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=5)
dt_depth_10 = DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=10)
dt_depth_5_leaf_10 = DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=5, min_samples_leaf=10)
dt_depth_5_leaf_20 = DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=5, min_samples_leaf=20)

# Random Forest models with different hyperparameters
rf_default = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_100_trees = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
rf_200_trees = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=200)
rf_depth_12 = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=12)
rf_optimized = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=200, max_depth=12, min_samples_leaf=5)

# Default SVM
svm_default = SVC(class_weight='balanced', random_state=42)

# Gradient Boosting models with different hyperparameters
gb_default = GradientBoostingClassifier(random_state=42)
gb_100_est = GradientBoostingClassifier(random_state=42, n_estimators=100)
gb_100_est_lr_3 = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.3)
gb_optimized = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

# Test all models
tester.test_model(classifier)
tester.test_model(dt_depth_5)
tester.test_model(dt_depth_10)
tester.test_model(dt_depth_5_leaf_10)
tester.test_model(dt_depth_5_leaf_20)
tester.test_model(rf_default)
tester.test_model(rf_100_trees)
tester.test_model(rf_200_trees)
tester.test_model(rf_depth_12)
tester.test_model(rf_optimized)
# tester.test_model(svm_default)
tester.test_model(gb_default)
tester.test_model(gb_100_est)
tester.test_model(gb_100_est_lr_3)
tester.test_model(gb_optimized)

--- Model: DecisionTreeClassifier ---
Precision: 28.50%
Recall:    29.51%
------------------------------
--- Model: DecisionTreeClassifier ---
Precision: 34.54%
Recall:    46.29%
------------------------------
--- Model: DecisionTreeClassifier ---
Precision: 30.57%
Recall:    56.10%
------------------------------
--- Model: DecisionTreeClassifier ---
Precision: 34.63%
Recall:    46.47%
------------------------------
--- Model: DecisionTreeClassifier ---
Precision: 34.72%
Recall:    46.75%
------------------------------
--- Model: RandomForestClassifier ---
Precision: 68.15%
Recall:    19.62%
------------------------------
--- Model: RandomForestClassifier ---
Precision: 68.15%
Recall:    19.62%
------------------------------
--- Model: RandomForestClassifier ---
Precision: 69.30%
Recall:    20.07%
------------------------------
--- Model: RandomForestClassifier ---
Precision: 39.41%
Recall:    55.09%
------------------------------
--- Model: RandomForestClassifier ---
Precision: 37.89%